In [23]:
import numpy as np
import pandas as pd
import os

In [24]:
df_path = os.path.join('data', 'Flight-Price.xlsx')

df = pd.read_excel(df_path)
df.head()

,Unnamed: 0,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [25]:
df = df.dropna()
df[['Date_of_Journey']] = df[['Date_of_Journey']].apply(pd.to_datetime)
df['day'] = df['Date_of_Journey'].dt.day.astype(int)
df['month'] = df['Date_of_Journey'].dt.month.astype(int)
df['Duration_min'] = df['Duration'].str.extract('(\d+)h').fillna(0).astype(int) * 60 + df['Duration'].str.extract('(\d+)m').fillna(0).astype(int)

df = df.drop(columns=['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10462
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       10462 non-null  int64  
 1   Airline          10462 non-null  object 
 2   Source           10462 non-null  object 
 3   Destination      10462 non-null  object 
 4   Total_Stops      10462 non-null  object 
 5   Additional_Info  10462 non-null  object 
 6   Price            10462 non-null  float64
 7   day              10462 non-null  int64  
 8   month            10462 non-null  int64  
 9   Duration_min     10462 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 899.1+ KB


C:\Users\ARITRA\AppData\Local\Temp\ipykernel_13108\907023303.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[['Date_of_Journey']] = df[['Date_of_Journey']].apply(pd.to_datetime)


In [26]:
# airports = ['Bangalore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai', 'New Delhi', 'Cochin', 'Hyderabad']
# stops = ['non-stop', '1 stop', '2 stops', '3 stops', '4 stops']
airports = {
	'Bangalore': 0,
	'Kolkata': 1,
	'Delhi': 2,
	'Chennai': 3,
	'Mumbai': 4,
	'New Delhi': 5,
	'Cochin': 6,
	'Hyderabad': 7
}

stops = {
	'non-stop': 0,
	'1 stop': 1,
	'2 stops': 2,
	'3 stops': 3,
	'4 stops': 4
}

In [27]:
df['Total_Stops'] = df['Total_Stops'].map(stops)

df.head()

,Unnamed: 0,Airline,Source,Destination,Total_Stops,Additional_Info,Price,day,month,Duration_min
0,0,IndiGo,Banglore,New Delhi,0,No info,3897.0,24,3,170
1,1,Air India,Kolkata,Banglore,2,No info,7662.0,1,5,445
2,2,Jet Airways,Delhi,Cochin,2,No info,13882.0,9,6,1140
3,3,IndiGo,Kolkata,Banglore,1,No info,6218.0,12,5,325
4,4,IndiGo,Banglore,New Delhi,1,No info,13302.0,1,3,285


In [28]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
le = LabelEncoder()

columns_to_encode = ['Airline', 'Additional_Info']
for column in columns_to_encode:
    le = LabelEncoder()
    le.fit(df[column])
    df[f"{column}_encoded"] = le.transform(df[column])
    print(f"Label values for column '{column}': {le.classes_}")



Label values for column 'Airline': ['Air Asia' 'Air India' 'GoAir' 'IndiGo' 'Jet Airways'
 'Jet Airways Business' 'Multiple carriers'
 'Multiple carriers Premium economy' 'SpiceJet' 'Trujet' 'Vistara'
 'Vistara Premium economy']
Label values for column 'Additional_Info': ['1 Long layover' '1 Short layover' '2 Long layover' 'Business class'
 'Change airports' 'In-flight meal not included' 'No Info'
 'No check-in baggage included' 'No info' 'Red-eye flight']


In [29]:
df.head()

,Unnamed: 0,Airline,Source,Destination,Total_Stops,Additional_Info,Price,day,month,Duration_min,Airline_encoded,Additional_Info_encoded
0,0,IndiGo,Banglore,New Delhi,0,No info,3897.0,24,3,170,3,8
1,1,Air India,Kolkata,Banglore,2,No info,7662.0,1,5,445,1,8
2,2,Jet Airways,Delhi,Cochin,2,No info,13882.0,9,6,1140,4,8
3,3,IndiGo,Kolkata,Banglore,1,No info,6218.0,12,5,325,3,8
4,4,IndiGo,Banglore,New Delhi,1,No info,13302.0,1,3,285,3,8
